## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import time
import numpy as np

# Import API keys
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Khandbari,NP,27.3747,87.2039,55.29,59,54,0.69,broken clouds
1,1,Marcali,HU,46.5850,17.4120,46.40,71,0,19.57,clear sky
2,2,Atuona,PF,-9.8000,-139.0333,78.22,77,59,15.43,broken clouds
3,3,Williston,US,48.1470,-103.6180,30.99,60,0,5.01,clear sky
4,4,Rikitea,PF,-23.1203,-134.9692,78.62,75,0,16.02,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria .
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 72
What is the maximum temperature you would like for your trip? 88


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Atuona,PF,-9.8000,-139.0333,78.22,77,59,15.43,broken clouds
4,4,Rikitea,PF,-23.1203,-134.9692,78.62,75,0,16.02,clear sky
5,5,Mahebourg,MU,-20.4081,57.7000,80.60,83,75,6.91,broken clouds
7,7,Vaini,TO,-21.2000,-175.2000,80.60,94,90,13.80,moderate rain
8,8,Nieuw Amsterdam,SR,5.8833,-55.0833,75.99,89,31,7.34,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
673,673,Prado,BR,-17.3411,-39.2208,74.50,86,25,4.25,scattered clouds
680,680,Kailua,US,21.4022,-157.7394,78.80,69,75,10.36,light rain
688,688,Mareeba,AU,-17.0000,145.4333,87.80,66,90,19.57,overcast clouds
690,690,Axim,GH,4.8699,-2.2405,77.85,89,27,4.32,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.  I decided to drop the whole row.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Atuona,PF,-9.8000,-139.0333,78.22,77,59,15.43,broken clouds
4,4,Rikitea,PF,-23.1203,-134.9692,78.62,75,0,16.02,clear sky
5,5,Mahebourg,MU,-20.4081,57.7000,80.60,83,75,6.91,broken clouds
7,7,Vaini,TO,-21.2000,-175.2000,80.60,94,90,13.80,moderate rain
8,8,Nieuw Amsterdam,SR,5.8833,-55.0833,75.99,89,31,7.34,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
673,673,Prado,BR,-17.3411,-39.2208,74.50,86,25,4.25,scattered clouds
680,680,Kailua,US,21.4022,-157.7394,78.80,69,75,10.36,light rain
688,688,Mareeba,AU,-17.0000,145.4333,87.80,66,90,19.57,overcast clouds
690,690,Axim,GH,4.8699,-2.2405,77.85,89,27,4.32,scattered clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Atuona,PF,78.22,broken clouds,-9.8000,-139.0333,
4,Rikitea,PF,78.62,clear sky,-23.1203,-134.9692,
5,Mahebourg,MU,80.60,broken clouds,-20.4081,57.7000,
7,Vaini,TO,80.60,moderate rain,-21.2000,-175.2000,
8,Nieuw Amsterdam,SR,75.99,scattered clouds,5.8833,-55.0833,
11,New Norfolk,AU,77.00,overcast clouds,-42.7826,147.0587,
14,Touros,BR,78.80,scattered clouds,-5.1989,-35.4608,
15,Georgetown,MY,82.40,few clouds,5.4112,100.3354,
16,Puerto Madryn,AR,73.51,broken clouds,-42.7692,-65.0385,
23,Loandjili,CG,78.51,light rain,-4.7561,11.8578,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [9]:
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # time.sleep just so I don't hit rate limits
    time.sleep(1.2)
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Atuona,PF,78.22,broken clouds,-9.8000,-139.0333,Villa Enata
4,Rikitea,PF,78.62,clear sky,-23.1203,-134.9692,Pension Maro'i
5,Mahebourg,MU,80.60,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
7,Vaini,TO,80.60,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
8,Nieuw Amsterdam,SR,75.99,scattered clouds,5.8833,-55.0833,Hotel Stardust
...,...,...,...,...,...,...,...
673,Prado,BR,74.50,scattered clouds,-17.3411,-39.2208,INN CORNER RIVER - PRADO - BAHIA - HOUSING AND...
680,Kailua,US,78.80,light rain,21.4022,-157.7394,Sheffield House Bed & Breakfast
688,Mareeba,AU,87.80,overcast clouds,-17.0000,145.4333,Trinity Plains Tourist Park
690,Axim,GH,77.85,scattered clouds,4.8699,-2.2405,AXIM BEACH RESORT AND SPA


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure.
fig

Figure(layout=FigureLayout(height='420px'))